In [2]:
import tensorflow as tf

# 简单介绍 

**我们把对训练集中的一个 Batch 运算更新一次叫做一个 Step，对训练集的所有样本循环迭代一次叫做一个 Epoch**
+ epochs：训练分为几个时期。每一个epoch是对整个输入数据的一次迭代（此操作以较小的批次完成）。
+ batch_size：当传递NumPy数据时，模型将数据切成较小的批次，并在训练期间对这些批次进行迭代。该整数指定每个批次的大小。请注意，如果不能将样本总数除以批次大小，则最后一批可能会更小。
+ validation_data：在模型训练时，监控在某些验证数据上监视其性能。传递此参数（输入和标签的元组）可以使模型在每个时期结束时以推断模式显示所传递数据的损失和度量。


In [70]:
x = tf.constant([1.])
w = tf.constant([2.])
# #括号中选择永久保存
with tf.GradientTape(persistent=True) as tape:
    #注意 [w] 中的w必须放在tape.watch()中.因为这个w不是tf.Variable型
    tape.watch([w])
    y1 = x + w
grad = tape.gradient(y1,[w])
  

In [71]:
    # 2nd-gradient
    b = tf.constant([3.])
    
try:
        with tf.GradientTape() as t1:
            with tf.GradientTape() as t2:
                y = x + w + b
            y_x = t2.gradient(y,[b])
        y_x2 = t1.gradient(y_x,[b])
        y_x2
        # y_w2 = t1.gradient(y_w,w)
except Exception as e :
        print(e)


'NoneType' object has no attribute 'dtype'


# 激活函数及其梯度
+ tf.sigmoid():x较大会出现偏导为0的情况(梯度消失)
+ tf.tanh()
+ tf.nn.relu() 减少梯度爆炸或者梯度下降的操作
   + tf.nn.relu6()

# 损失函数及其梯度
+ tf.reduce_mean(tf.losses.MSE(y,out)) 
+ tf.losses.categorical_crossentropy(y,out):softmax

# 单输出感知机及其梯度 
+ MSE中的参数y_pred必须是tf.float32型

In [15]:
x = tf.random.normal([1,3])
b = tf.ones([1])
w = tf.ones([3,1])
y = tf.constant([1])

with tf.GradientTape() as tape:
    tape.watch([w,b])
    prob = tf.sigmoid(x @ w + b)    
    loss = tf.reduce_mean(tf.losses.MSE(prob,y))
 
grads = tape.gradient(loss,[w,b])
grads[0]

<tf.Tensor: id=244, shape=(3, 1), dtype=float32, numpy=
array([[-0.01808497],
       [ 0.01421455],
       [-0.0117125 ]], dtype=float32)>

# 多输出感知机及其梯度

In [16]:
x = tf.random.normal([2,4])
b = tf.ones([3])
w = tf.ones([4,3])
y = tf.constant([2,0])

with tf.GradientTape() as tape:
    tape.watch([w,b])
    #axis=1 表示结果[b,3]中的3这个维度为概率
    prob = tf.nn.softmax(x @ w + b,axis=1)    
    loss = tf.reduce_mean(tf.losses.MSE(tf.one_hot(y,depth=3),prob))
 
grads = tape.gradient(loss,[w,b])
grads[0]

<tf.Tensor: id=304, shape=(4, 3), dtype=float32, numpy=
array([[ 0.08003316, -0.04320253, -0.03683063],
       [ 0.11648539, -0.02440556, -0.09207983],
       [ 0.1069216 , -0.11589715,  0.00897555],
       [ 0.11371143,  0.04293453, -0.15664595]], dtype=float32)>

# 手写数字识别实战
见py文件

# TensorBorad可视化

##  优化一

In [14]:
with tf.GradientTape() as tape:
    x = tf.reshape(x, (-1, 28*28))
    out = network(x)
    y_onehot = tf.one_hot(y, depth=10)
    loss = tf.reduce_mean(tf.losses.categorical_crossentropy(y_onehot, out, from_logits=True))
    
grads = tape.gradient(loss, network.trainable_variables)
optimizer.apply_gradients(zip(grads, network.trainable_variables))


NameError: name 'x' is not defined

上面一段代码可以简化为：

In [15]:
network.compile(optimizer=optimizers.Adam(lr=0.01),
                loss=tf.losses.CategoricalCrossentropy(fromlogits=True),
                metircs=['accuracy'])

network.fit(db, epochs=10)
#上面等同于下一句
# for epoch in range(epochs):
#     for step, (x, y) in enumerate(db):
#         # ...

NameError: name 'network' is not defined

## 优化二 

In [ ]:
# if step % 500 == 0:
#     total, total_correct = 0., 0
    
#     for step, (x, y) in enumerate(ds_val):
#         x = tf.reshape(x, (-1, 28*28))
#         out = network(x)
#         pred = tf.argmax(out, axis=1)
#         pred = tf.cast(pred, dtype=tf.int32)
#         correct = tf.equal(pred, y)
#         total_correct += tf.reduce_sum(tf.cast(correct, dtype=tf.int32)).numpy()
#         total += x.shape[0]
       
#     print(step, 'Evaluate Acc:', total_correct/total)


network.compile(optimizer=optimizers.Adam(lr=0.01),
                loss=tf.losses.CategoricalCrossentropy(fromlogits=True),
                metircs=['accuracy'])

# validation_freq=2表示2个epochs做一次验证
network.fit(db, epochs=10, validation_data=ds_val, validation_freq=2)

#测试
network.evaluate(ds_val)

In [ ]:
#predict
sample = next(iter(ds_val))
x = sample[0]
y = sample[1]
pred = network.predict(x)
y = tf.argmax(y, axis=1)
pred = tf.argmax(pre, axis=1)

print(pred)
print(y)